# Regresión logística

La idea de este notebook es realizar una predicción mediante regresión logística utilizando los preprocessings:
* Standard preprocessing parte 1
* Preprocessing significantes 90% de varianza explicada
* Primeras variables seleccionadas por un árbol de decisión.

In [ ]:
# Modelo
from sklearn.linear_model import LogisticRegression

# Preprocessing
from preprocessing import standard_preprocessing_base_parte_1, preprocessing_significantes, preprocessing_mejores_por_arbol
import utils
from sklearn.preprocessing import StandardScaler

# Búsqueda
from sklearn.model_selection import train_test_split, GridSearchCV

# Metrics
from sklearn.metrics import plot_roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report, plot_confusion_matrix

# Plots
from matplotlib import pyplot as plt
import seaborn as sns

# Otros
import pandas as pd
import numpy as np

In [ ]:
df = utils.get_data()

## Modelo 1 - Standard Preprocessing parte 1

Será entrenado con las siguientes variables:

* **Ganancia/Pérdida declara en la bolsa argentina**: `ganancia_perdida_declarada_bolsa_argentina`
* **Edad**: `edad`
* **Rol familiar registrado**: `rol_familiar_registrado`
* **Años estudiados**: `anios_estudiados`

A las que se les aplicará una función de Standarización para que la regresión logísitca y su regularización funcionen correctamente.

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
Y = df['tiene_alto_valor_adquisitivo']

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, Y, random_state=112)

X_train_1, X_test_1 = standard_preprocessing_base_parte_1(X_train_1, X_test_1)

In [ ]:
model = LogisticRegression()
params = [{'penalty': ['l1']        , 'C': np.logspace(-3, 3, 13), 'solver': ['saga'] },
          {'penalty': ['l2']        , 'C': np.logspace(-3, 3, 13), 'solver': ['lbfgs']},
          {'penalty': ['elasticnet'], 'C': np.logspace(-3, 3, 13), 'solver': ['saga'] , 'l1_ratio': np.linspace(0.1, 0.9, 9)}]

gscv1 = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, verbose=4, cv=10)

In [ ]:
gscv1.fit(X_train_1, y_train_1)

In [ ]:
print("Los parámetros que mejor score obtuvieron para AdaBoost fueron:", gscv1.best_params_)
print("Con un Score (RocAUC) de: ", round(gscv1.best_score_, 4))

Realicemos ahora el modelo por fuera del gridSearch

In [ ]:
lr1 = gscv1.best_estimator_

In [ ]:
lr1.fit(X_train_1, y_train_1)

In [ ]:
print(classification_report(y_test_1,lr1.predict(X_test_1)))

Vemos que el modelo tiene un accuracy de 84%. Un recall común para los ceros, un poco pobre para los unos, esto se verá reflejado en la matriz de confusión

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(lr1, X_test_1, y_test_1, cmap=plt.cm.Blues, display_labels=['0', '1'], ax=ax)
plt.show()

Vemos aquí que hay muchos Falsos negativos, es decir que el modelo no está pudiendo predecir correctamente a los unos.

In [ ]:
plot_roc_curve(lr1, X_test_1, y_test_1)

In [ ]:
roc_auc_score(y_test_1, lr1.predict_proba(X_test_1)[:,1])

Vemos que el RocAUC fue de 0.88, por lo que estamos frente a un modelo decente, pero definitivamente no el mejor.

## Modelo 2 - Preprocessing Significantes 90% varianza explicada

Queremos ver ahora cómo actúa la regresión logística con las variables proyectadas por PCA con 90% de la varianza explicada.


In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
X = pd.get_dummies(X, drop_first=True)
Y = df['tiene_alto_valor_adquisitivo']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, Y, random_state=112)

X_train_2, X_test_2 = preprocessing_significantes(X_train_2, X_test_2, 0.9)

In [ ]:
model = LogisticRegression()
params = [{'penalty': ['l1']        , 'C': np.logspace(-3, 3, 7), 'solver': ['saga'] },
          {'penalty': ['l2']        , 'C': np.logspace(-3, 3, 7), 'solver': ['lbfgs']},
          {'penalty': ['elasticnet'], 'C': np.logspace(-3, 3, 7), 'solver': ['saga'] , 'l1_ratio': np.linspace(0.1, 0.9, 9)}]

gscv2 = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, verbose=4, cv=10)

In [ ]:
gscv2.fit(X_train_2, y_train_2)

In [ ]:
print("Los parámetros que mejor score obtuvieron para AdaBoost fueron:", gscv2.best_params_)
print("Con un Score (RocAUC) de: ", round(gscv2.best_score_, 4))

In [ ]:
lr2 = gscv2.best_estimator_

In [ ]:
lr2.fit(X_train_2, y_train_2)

In [ ]:
print(classification_report(y_test_2,lr2.predict(X_test_2)))

Vemos que el modelo obtenido posee métricas similares al anterior modelo.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(lr2, X_test_2, y_test_2, cmap=plt.cm.Blues, display_labels=['0', '1'], ax=ax)
plt.show()

La matriz de confusión tamibén es similar a la obtenida por el modelo anterior

In [ ]:
plot_roc_curve(lr2, X_test_2, y_test_2)

In [ ]:
roc_auc_score(y_test_2, lr2.predict_proba(X_test_2)[:,1])

La curva ROC y su puntuación AUC son decentes, pero podemos ver que no se trata del mejor modelo.

## Modelo 3 - mejores variables del árbol

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
X = pd.get_dummies(X)
Y = df['tiene_alto_valor_adquisitivo']

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, Y, random_state=27)

X_train_3, X_test_3 = preprocessing.preprocessing_4_mejores_variables_arbol(X_train, X_test)

### El árbol de decisión. Veamos qué variables elegimos

In [ ]:
import graphviz
import dtreeviz.trees as dtreeviz

In [ ]:
arbol = DecisionTreeClassifier(min_samples_leaf=300, max_depth=5, random_state=27)
arbol.fit(X_train, y_train)

In [ ]:
viz = dtreeviz.dtreeviz(
    arbol,
    X_train,
    y_train,
    target_name='tiene_alto_valor_adquisitivo',
    feature_names=list(X.columns),
    class_names=list([0,1]),
    scale=1.5,
)

display(viz)

##### Variables seleccionadas:
(La selección la hacemos nosotors mirando las ganancias en separación con los pieplots)

* Rol familiar registrado.
* Años estudiados.
* ganancia/perdida declarada en la bolsa.
* Edad.
* horas de trabajo registradas.

Por lo que se usarán dichas variables para realizar la regresión, además se escalarán para poder aplicar regularización.

In [ ]:
df['horas_trabajo_registradas']

In [ ]:
def filtrar_variables(X):
    seleccion = ['rol_familiar_registrado_casado', 'anios_estudiados', 'ganancia_perdida_declarada_bolsa_argentina', 'edad', 'horas_trabajo_registradas']
    return X[seleccion]

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
X = pd.get_dummies(X)
X = filtrar_variables(X)
Y = df['tiene_alto_valor_adquisitivo']

# El mismo random_state nos asegura no estar provocando un leaking al seleccionar las variables.
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=27)

## Entrenamiento del modelo

##### Primero se escalan las features

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

X_train = pd.DataFrame(scaler.fit_transform(X_train)).set_axis(X_train.columns, axis=1)
X_test = pd.DataFrame(scaler.transform(X_test)).set_axis(X_train.columns, axis=1)

##### Búsqueda de hiperparámetros con KFoldCV

In [ ]:
model = LogisticRegression(n_jobs=-1, random_state=27)
params = [{'penalty': ['l1']        , 'C': np.logspace(-3, 2, 6), 'solver': ['saga'] },
          {'penalty': ['l2']        , 'C': np.logspace(-3, 2, 6), 'solver': ['lbfgs']},
          {'penalty': ['elasticnet'], 'C': np.logspace(-3, 2, 6), 'solver': ['saga'] , 'l1_ratio': np.linspace(0.1, 0.9, 9)}]

gscv = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, cv=10, verbose=4)

In [ ]:
gscv.fit(X_train, y_train)

In [ ]:
print(gscv.best_estimator_)
print(gscv.best_params_)
print(gscv.best_score_)

In [ ]:
print(classification_report(y_test,gscv.predict(X_test)))

##### El modelo

In [ ]:
modelo = gscv.best_estimator_

modelo.fit(X_train, y_train)

In [ ]:
modelo.coef_

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
plot_roc_curve(modelo, X_test, y_test, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4), dpi=150)
plt.grid(False)
plot_confusion_matrix(modelo, X_test, y_test, cmap=plt.cm.Blues, ax=ax)
plt.show()